In [4]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

Note: you may need to restart the kernel to use updated packages.
NumExpr defaulting to 8 threads.
⠧ Installing integrations.....
Note: you may need to restart the kernel to use updated packages.


In [5]:
IPython.Application.instance().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [6]:
#ZenML setup

!rm -rf .zen
!zenml init

NumExpr defaulting to 8 threads.
⠋ Initializing ZenML repository at /Users/apple/Downloads/ML projects/MLOps.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /Users/apple/Downloads/ML projects/MLOps.
⠋ Initializing ZenML repository at /Users/apple/Downloads/ML projects/MLOps.
⠋ Initializing ZenML repository at /Users/apple/Downloads/ML projects/MLOps.

The local active stack was initialized to 'default'. This local configuration 
will only take effect when you're running ZenML from the initialized repository 
root, or from a subdirectory. For more information on repositories and 
configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


In [4]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images),-1))
    X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size = 0.2, shuffle=False)
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    

train_test()

Test accuracy: 0.9583333333333334


In [11]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd 
from typing import Tuple
import numpy as np

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """Load the digits dataset as numpy arrays"""
    digits = load_digits()
    data = digits.data.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size = 0.2, shuffle=False
    )
    
    return X_train, X_test, y_train, y_test


@step
def svc_trainer(
 X_train : np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn svc classifier """
    model = SVC(gamma = 0.001)
    model.fit(X_train, y_train)
    return model

@step 
def evaluator(
    X_test : np.ndarray,
    y_test : np.ndarray,
        model:ClassifierMixin,
) -> float :
        """Calculate the test set accuracy of an sklearn model. """
        test_acc = model.score(X_test, y_test)
        print(f"Test Accuracy : {test_acc}")
        return test_acc

    

        

In [12]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    """Links all the steps together in pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train=X_train, y_train = y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

In [13]:
digits_svc_pipeline = digits_pipeline()


Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 3).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Step evaluator has started.
Test Accuracy : 0.9583333333333334
Step evaluator has finished in 0.547s.
Pipeline run has finished in 0.660s.
